In [1]:
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import pandas as pd
from langchain.chains import LLMChain
from langfuse.callback import CallbackHandler
from langchain.llms import OpenAI
import os
from langchain.chains import SequentialChain


load_dotenv()

handler = CallbackHandler(os.environ.get("LANGFUSE_PUBLIC_KEY"), os.environ.get("LANGFUSE_SECRET_KEY"))

In [2]:
#MODEL = "gpt-3.5-turbo"
MODEL = "gpt-4-1106-preview"

OUTPUT_FOLDER = "../generated_GUIs/"
IMAGE_INPUT_FOLDER = "../data/"
S2W = "../data/Screen2Words.csv"
S2W_SUMMARIZED = "../data/s2w_summarized.csv"
S2W_SAMPLE = "../data/s2w_sample.csv"
METADATA = "../data/metadata.csv"

In [3]:
s2w = pd.read_csv(S2W)
s2w.drop(columns=["Unnamed: 0"], inplace=True)

s2w_summarized = pd.read_csv(S2W_SUMMARIZED)
s2w_summarized.drop(columns=["Unnamed: 0"], inplace=True)

s2w_sample = pd.read_csv(S2W_SAMPLE)
s2w_sample.drop(columns=["Unnamed: 0"], inplace=True)

rico = pd.read_csv(METADATA)
rico.drop(columns=["Unnamed: 0"], inplace=True)

In [4]:
from prompts import SUMMARY2REQUIREMENTS_ZS, REQUIREMENTS2IMPLEMENTATION_ZS, IMPLEMETATION2STRUCTURE_ZS, STRUCTURE2CODE_ZS

def s2r2e2s2c (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIREMENTS_ZS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_ZS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_ZS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))


In [5]:
from prompts import SUMMARY2REQUIRMENTS_FS, REQUIREMENTS2IMPLEMENTATION_FS, IMPLEMETATION2STRUCTURE_FS, STRUCTURE2CODE_FS

def s2r2e2s2c_fs (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2REQUIRMENTS_FS, output_key="requirements")
    chain_2 = LLMChain(llm=llm, prompt=REQUIREMENTS2IMPLEMENTATION_FS, output_key="implementation")
    chain_3 = LLMChain(llm=llm, prompt=IMPLEMETATION2STRUCTURE_FS, output_key="structure")
    chain_4 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code")

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2,chain_3,chain_4],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [7]:
from prompts import SUMMARY2STRUCTURE_ZS, STRUCTURE2CODE_ZS

def s2s2c (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2STRUCTURE_ZS, output_key="structure",)
    chain_2 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_ZS, output_key="code",)

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [4]:
from prompts import SUMMARY2STRUCTURE_FS, STRUCTURE2CODE_FS

def s2s2c_fs (summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain_1 = LLMChain(llm=llm, prompt=SUMMARY2STRUCTURE_FS, output_key="structure",)
    chain_2 = LLMChain(llm=llm, prompt=STRUCTURE2CODE_FS, output_key="code",)

    seq_chain = SequentialChain(
        chains = [chain_1,chain_2],
        input_variables=["summary"])
    
    return(seq_chain.run(summary, callbacks=[handler]))

In [4]:
from prompts import SUMMARY2CODE_ZS

def s2c(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain = LLMChain(llm=llm, prompt=SUMMARY2CODE_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [11]:
from prompts import ALLINONEPROMPT_ZS

def s2c_xot_zs(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain = LLMChain(llm=llm, prompt=ALLINONEPROMPT_ZS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [4]:
from prompts import ALLINONEPROMPT_FS

def s2c_xot_fs(summary, temp=0.7):
    llm = ChatOpenAI(model_name=MODEL,temperature=temp, max_tokens=1200)
    chain = LLMChain(llm=llm, prompt=ALLINONEPROMPT_FS, callbacks=[handler])

    try: 
        response = chain.run(summary=summary, callbacks=[handler])
    except Exception as e:
        print(e)
    
    return(response)

In [12]:
def pipeline(summary):
    
    response = s2c(summary)
    try: 
        with open("instruction.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("Failed")
        print(e)
    
    response = s2c_xot_zs(summary)
    try: 
        with open("xot_zs.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("Failed")
        print(e)

    response = s2c_xot_fs(summary)
    try: 
        with open("xot_fs.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("Failed")
        print(e)
    
    response = s2s2c(summary)
    try: 
        with open("2_step_zs.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("Failed")
        print(e)
    
    response = s2s2c_fs(summary)
    try: 
        with open("2_step_fs.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("Failed")
        print(e)
    
    response = s2r2e2s2c(summary)
    try: 
        with open("question_decomposition_zs.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("Failed")
        print(e)
    
    response = s2r2e2s2c_fs(summary)
    try: 
        with open("question_decomposition_fs.html", 'w') as f:
            f.write(response)
    except Exception as e:
        print("Failed")
        print(e)


In [13]:
pipeline("The screen displays a track playing in a music application, along with various options")